In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = input()

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
mistralai/Mistral-7B-v0.1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [16]:
import torch
from torch.optim import Adam
from transformers.optimization import get_linear_schedule_with_warmup
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import time


max_length = 50
num_return_sequences = 1
batch_size = 32

def generate_text(model, tokenizer, prompt, max_length=50, num_return_sequences=1):
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=num_return_sequences,
                             temperature=0.7, top_k=50, top_p=0.95, repetition_penalty=1.0)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

def calculate_bleu_score(candidate, references):
    candidate = [candidate.split()]
    references = [[reference.split()] for reference in references]
    return corpus_bleu(references, candidate, smoothing_function=SmoothingFunction().method1)

def calculate_metrics(model_name, tokenizer_name, prompt):
    model = model_name
    tokenizer = tokenizer_name
    generated_text = generate_text(model, tokenizer, prompt)[0]
    reference_text = [prompt]
    bleu_score = calculate_bleu_score(generated_text, reference_text)
    print(f"\nBLEU Score: {bleu_score:.4f}")

    return bleu_score

def measure_inference_performance(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    start_time = time.time()
    outputs = model.generate(input_ids, max_length=128, num_return_sequences=1)
    end_time = time.time()
    inference_time = end_time - start_time
    tokens_per_second = (128 + 128) / inference_time
    print("Response:", tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("Inference Time:", inference_time)
    print("Tokens Per Second:", tokens_per_second)

if __name__ == "__main__":
    optimizer = Adam(model.parameters(), lr=1e-5)
    num_warmup_steps = 1000
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)
    prompt = input()
    calculate_metrics(model, tokenizer, prompt)
    measure_inference_performance(prompt)

Hi my name is Shaurya Sarswat and I have been coding in this language for a few years that is cpp


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1482: UserWarning: You are calling .generate() with the `input_ids` being o


BLEU Score: 0.4677
Response: Hi my name is Shaurya Sarswat and I have been coding in this language for a few years that is cpp. I have been using it for a few years and I have been using it for a few years. I have been using it for a few years and I have been using it for a few years. I have been using it for a few years and I have been using it for a few years. I have been using it for a few years and I have been using it for a few years. I have been using it for a few years and I have been using it for a few years
Inference Time: 28.044981479644775
Tokens Per Second: 9.128192870649832
